### Packages

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',1000)

In [8]:
dataset = pd.read_csv('labeledTrainData.tsv',sep='\t')

In [9]:
dataset.head()

,id,sentiment,review
0,5814_8,1,"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally ..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \""critics\"" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \""critics\"". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \""critics\"" perceive to be its shortcomings."""
2,7759_3,0,"The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like ¨Jurassik Park¨, and some scientists resurrect one of nature's most fearsome predators, the Sabretooth tiger or Smilodon . Scientific ambition turns deadly, however, and when the high voltage fence is opened the creature escape and begins savagely stalking its prey - the human visitors , tourists and scientific.Meanwhile some youngsters enter in the restricted area of the security center and are attacked by a pack of large pre-historical animals which are deadlier and bigger . In addition , a security agent (Stacy Haiduk) and her mate (Brian Wimmer) fight hardly against the carnivorous Smilodons. The Sabretooths, themselves , of course, are the real star stars and they are astounding terrifyingly though not convincing. The giant animals savagely are stalking its prey and the group run afoul and fight against..."
3,3630_4,0,"It must be assumed that those who praised this film (\the greatest filmed opera ever,\"" didn't I read somewhere?) either don't care for opera, don't care for Wagner, or don't care about anything except their desire to appear Cultured. Either as a representation of Wagner's swan-song, or as a movie, this strikes me as an unmitigated disaster, with a leaden reading of the score matched to a tricksy, lugubrious realisation of the text.<br /><br />It's questionable that people with ideas as to what an opera (or, for that matter, a play, especially one by Shakespeare) is \""about\"" should be allowed anywhere near a theatre or film studio; Syberberg, very fashionably, but without the smallest justification from Wagner's text, decided that Parsifal is \""about\"" bisexual integration, so that the title character, in the latter stages, transmutes into a kind of beatnik babe, though one who continues to sing high tenor -- few if any of the actors in the film are the

### Step 1 Need to clean the review column

In [85]:
def get_tokens(para):
    from nltk.tokenize import sent_tokenize
    from nltk.corpus import stopwords
    stp = stopwords.words('english')
    import json
    with open('word_contractions.json') as f:
        word_contractions = json.load(f)

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(para, "html.parser")
    stripped_text = soup.get_text()
    alltokens =[]

    for sent in sent_tokenize(stripped_text):
        wrds = [wrd.lower() for wrd in sent.split(' ')]
        new_wrds =[]

        for i in wrds:           
            if i.find("'")>0:
                new_wrds.extend(word_contractions.get(i,i[:i.index("'")]).split())
            else:
                temp = [wds for wds in i.split('.') if len(wds)>0]            
                if len(temp)==2 and len(temp[0])>2 and len(temp[1])>0:
                    new_wrds.extend(temp)
                    continue



                new_wrds.append(i.translate ({ord(c): "" for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"}))

        new_wrds = [wrd for wrd in new_wrds if wrd not in stp and wrd.isdigit()!=True]
        alltokens.extend(new_wrds)
        
    return ','.join(alltokens)

#### Test the function 

In [86]:
print(get_tokens(dataset['review'][0]))

stuff,going,moment,mj,started,listening,music,watching,odd,documentary,watched,wiz,watched,moonwalker,maybe,want,get,certain,insight,guy,thought,really,cool,eighties,maybe,make,mind,whether,guilty,innocent,moonwalker,part,biography,part,feature,film,remember,going,see,cinema,originally,released,subtle,messages,mj,feeling,towards,press,also,obvious,message,drugs,bad,impressive,course,michael,jackson,unless,remotely,like,mj,anyway,going,hate,find,boring,may,call,mj,egotist,consenting,making,movie,mj,fans,would,say,made,fans,true,really,nice,actual,feature,film,bit,finally,starts,minutes,excluding,smooth,criminal,sequence,joe,pesci,convincing,psychopathic,powerful,drug,lord,wants,mj,dead,bad,beyond,mj,overheard,plans,nah,joe,pesci,character,ranted,wanted,people,know,supplying,drugs,etc,dunno,maybe,hates,mj,music,lots,cool,things,like,mj,turning,car,robot,whole,speed,demon,sequence,also,director,must,patience,saint,came,filming,kiddy,bad,sequence,usually,directors,hate,working,one,kid,let,

### Step 2 Create a new column of review Tokens

In [87]:
#apply functions are very fast as compared to manual iteration using iterrows

dataset['review_tokens'] = dataset.review.apply(lambda x:get_tokens(x))

### Step 3 Check label balance

In [138]:
dataset.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

### Step 4 Split the Data into Train and Test

In [100]:
X=dataset.review_tokens
y=dataset.sentiment

In [101]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123)

In [102]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((17500,), (17500,), (7500,), (7500,))

### Step 5 Creating a Baseline Model -Bag of Words +Naive-Bayes

In [125]:

from sklearn.feature_extraction.text import CountVectorizer

# build BOW features on train reviews
cv = CountVectorizer(binary=False, ngram_range=(1,2)) #Just settnig minimum doc freq to 2 helps cut the vocab considerably
cv_train_features = cv.fit_transform(X_train)

In [126]:
cv_train_features.shape

(17500, 1418114)

In [140]:
cv_test_features = cv.transform(X_test)

In [132]:
cv_test_features.shape

(7500, 1418114)

In [149]:
# cv.inverse_transform(cv_test_features)

In [128]:
cv.vocabulary_

{'already': 40945,
 'put': 982806,
 'best': 121410,
 'version': 1333084,
 'ever': 398707,
 'homer': 585633,
 'epic': 386167,
 'entirely': 385110,
 'shot': 1118597,
 'natural': 836241,
 'locations': 723362,
 'mediterranean': 776776,
 'sea': 1082171,
 'sky': 1139578,
 'strikingly': 1196978,
 'blue': 137142,
 'islands': 640207,
 'green': 540091,
 'untouched': 1319277,
 'clothing': 224072,
 'linen': 714865,
 'wool': 1391463,
 'fur': 496283,
 'settings': 1106946,
 'stony': 1187705,
 'bare': 102477,
 'everything': 401773,
 'somewhat': 1154433,
 'rugged': 1052751,
 'primitive': 964946,
 'bit': 129760,
 'would': 1399834,
 'find': 459574,
 'cacoyannis': 167621,
 'pasolini': 905426,
 'movies': 818134,
 'makes': 751312,
 'authentic': 87471,
 'although': 44552,
 'story': 1189741,
 'based': 103959,
 'myths': 831731,
 'widely': 1377554,
 'goes': 523922,
 'supernatural': 1213108,
 'gives': 518374,
 'us': 1321804,
 'good': 527437,
 'idea': 603679,
 'life': 702655,
 '10th': 353,
 'century': 190404,
 'b

In [139]:
from sklearn.naive_bayes import BernoulliNB

nb = BernoulliNB()

nb.fit(cv_train_features,y_train.values)

D:\Python\lib\site-packages\sklearn\naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


BernoulliNB(alpha=0, binarize=0.0, class_prior=None, fit_prior=True)

In [141]:
nb_predict_train1 =  nb.predict(cv_train_features)

In [142]:
nb_predict1 = nb.predict(cv_test_features)

In [144]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

print("Training Metrics")
print("Accuracy:",accuracy_score(y_train,nb_predict_train1))
print("Precision:",precision_score(y_train,nb_predict_train1))
print("Recall:",recall_score(y_train,nb_predict_train1))


print("Test Metrics")
print("Accuracy:",accuracy_score(y_test,nb_predict1))
print("Precision:",precision_score(y_test,nb_predict1))
print("Recall:",recall_score(y_test,nb_predict1))


from sklearn.metrics import classification_report
print('Test Classification Report')
print(classification_report(y_test,nb_predict1))

from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, nb_predict1)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('Roc -Auc Curve',roc_auc)

Training Metrics
Accuracy: 0.9999428571428571
Precision: 1.0
Recall: 0.999885452462772
Test Metrics
Accuracy: 0.8192
Precision: 0.8294213973799127
Recall: 0.8061007957559682
Test Classification Report
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      3730
           1       0.83      0.81      0.82      3770

   micro avg       0.82      0.82      0.82      7500
   macro avg       0.82      0.82      0.82      7500
weighted avg       0.82      0.82      0.82      7500

Roc -Auc Curve 0.8192702370200753


### Step 5 Creating a Baseline Model Using TF-IDF + Naive Bayes

In [150]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [151]:
tfidf = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0, ngram_range=(1,2),sublinear_tf=True)

tv_train_features = tfidf.fit_transform(X_train)

In [152]:
tv_train_features.shape

(17500, 1418114)

In [153]:
tv_test_features = tfidf.transform(X_test)

tv_test_features.shape

(7500, 1418114)

In [154]:
from sklearn.naive_bayes import BernoulliNB

nb2 = BernoulliNB()

nb2.fit(tv_train_features,y_train.values)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [155]:
nb_predict_train2 =  nb2.predict(tv_train_features)

In [156]:
nb_predict2 = nb.predict(tv_test_features)

In [157]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

print("Training Metrics")
print("Accuracy:",accuracy_score(y_train,nb_predict_train2))
print("Precision:",precision_score(y_train,nb_predict_train2))
print("Recall:",recall_score(y_train,nb_predict_train2))


print("Test Metrics")
print("Accuracy:",accuracy_score(y_test,nb_predict2))
print("Precision:",precision_score(y_test,nb_predict2))
print("Recall:",recall_score(y_test,nb_predict2))


from sklearn.metrics import classification_report
print('Test Classification Report')
print(classification_report(y_test,nb_predict2))

from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, nb_predict2)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('Roc -Auc Curve',roc_auc)

Training Metrics
Accuracy: 0.9956571428571429
Precision: 1.0
Recall: 0.9912943871706759
Test Metrics
Accuracy: 0.8192
Precision: 0.8294213973799127
Recall: 0.8061007957559682
Test Classification Report
              precision    recall  f1-score   support

           0       0.81      0.83      0.82      3730
           1       0.83      0.81      0.82      3770

   micro avg       0.82      0.82      0.82      7500
   macro avg       0.82      0.82      0.82      7500
weighted avg       0.82      0.82      0.82      7500

Roc -Auc Curve 0.8192702370200753


### Step 6 TFIDF + Random Forest

In [234]:
from sklearn.ensemble import RandomForestClassifier

# rf_model2 = RandomForestClassifier(random_state=123,n_estimators=1000,max_depth=10)

rf_model2 = RandomForestClassifier(random_state=123,n_estimators=1000,max_depth=10,max_leaf_nodes=50)

rf_model2.fit(tv_train_features,y_train)

rf_train_pred2= rf_model2.predict(tv_train_features)

rf_test_pred2 = rf_model2.predict(tv_test_features)

In [235]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

print("Test Metrics")
print("Accuracy:",accuracy_score(y_test,rf_test_pred2))
print("Precision:",precision_score(y_test,rf_test_pred2))
print("Recall:",recall_score(y_test,rf_test_pred2))


from sklearn.metrics import classification_report
print('Test Classification Report')
print(classification_report(y_test,rf_test_pred2))

from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, rf_test_pred2)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('Roc -Auc Curve',roc_auc)

Test Metrics
Accuracy: 0.8616
Precision: 0.86004217185029
Recall: 0.8655172413793103
Test Classification Report
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      3730
           1       0.86      0.87      0.86      3770

   micro avg       0.86      0.86      0.86      7500
   macro avg       0.86      0.86      0.86      7500
weighted avg       0.86      0.86      0.86      7500

Roc -Auc Curve 0.8615789960247758


### Step 6 TFIDF +  LightGBM

In [158]:
!pip install lightgbm

You are using pip version 9.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [192]:
import lightgbm as lgb
d_train = lgb.Dataset(tv_train_features, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
# params['sub_feature'] = 0.5
# params['num_leaves'] = 10
# params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 2500)

In [193]:
#Prediction
y_pred_gbm_train=clf.predict(tv_train_features)

#convert into binary values
for i in range(0,17500):
    if y_pred_gbm_train[i]>=.5:
        y_pred_gbm_train[i]=1
    else:
        y_pred_gbm_train[i]=0

In [194]:
y_pred_gbm_test =clf.predict(tv_test_features)

#convert into binary values
for i in range(0,7500):
    if y_pred_gbm_test[i]>=.5:
        y_pred_gbm_test[i]=1
    else:
        y_pred_gbm_test[i]=0

In [195]:
from sklearn.metrics import accuracy_score,precision_score,recall_score

print("Test Metrics")
print("Accuracy:",accuracy_score(y_test,y_pred_gbm_test))
print("Precision:",precision_score(y_test,y_pred_gbm_test))
print("Recall:",recall_score(y_test,y_pred_gbm_test))


from sklearn.metrics import classification_report
print('Test Classification Report')
print(classification_report(y_test,y_pred_gbm_test))

from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_gbm_test)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('Roc -Auc Curve',roc_auc)

Test Metrics
Accuracy: 0.8356
Precision: 0.8288825512056002
Recall: 0.8480106100795756
Test Classification Report
              precision    recall  f1-score   support

           0       0.84      0.82      0.83      3730
           1       0.83      0.85      0.84      3770

   micro avg       0.84      0.84      0.84      7500
   macro avg       0.84      0.84      0.84      7500
weighted avg       0.84      0.84      0.84      7500

Roc -Auc Curve 0.8355334551738361


### Step 7 Choosing the best Model to work with Test data(Kaggle)

#### Load the test data

In [196]:
testdata =pd.read_csv('testData.tsv',sep='\t')

In [198]:
testdata.shape

(25000, 2)

In [200]:
testdata.head(1)

,id,review
0,12311_10,"Naturally in a film who's main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce's short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."


In [201]:
testdata['review_tokens'] = testdata.review.apply(lambda x:get_tokens(x))

In [203]:
tfidf_test_features = tfidf.transform(testdata.review_tokens)

tfidf_test_features.shape

(25000, 1418114)

In [204]:
rf_test_predict = rf_model2.predict(tfidf_test_features)

In [206]:
Predictions =pd.Series(rf_test_predict)

In [210]:
Predictions.head()

0    1
1    0
2    1
3    1
4    1
dtype: int64

In [215]:
testdata.id.head()

0    12311_10
1      8348_2
2      5828_4
3      7186_2
4     12128_7
Name: id, dtype: object

In [225]:
f1 =testdata['id'].to_frame()
f2 = Predictions.to_frame()

In [227]:
submission = f1.join(f2)

In [229]:
submission.columns = ['id', 'Sentiment']

In [233]:
submission.to_csv('Final_submission.csv',index = False)